In [1]:
import pandas as pd
import numpy as np
import re
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as md
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import psutil

# 检查可用内存
memory_info = psutil.virtual_memory()
print(f'Available Memory: {memory_info.available / (1024 ** 3):.2f} GB')

Available Memory: 2.38 GB


In [3]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                                    stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.utils.weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                                    stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [4]:
class TCNClassifier(nn.Module):
    def __init__(self, input_size, num_classes, num_channels, kernel_size=2, dropout=0.2):
        super(TCNClassifier, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        y1 = self.tcn(x)
        o = self.fc(y1[:, :, -1])
        return o

In [5]:
if torch.cuda.is_available():
    print("true")
else:
    print("false")

false


In [6]:
file_path = 'result/output.csv'
# 分批加载数据
# chunk_size = 10000 
# data = pd.read_csv(file_path,nrows=100)
data = pd.read_csv(file_path)

In [7]:
memory_info = psutil.virtual_memory()
print(f'Available Memory: {memory_info.available / (1024 ** 3):.2f} GB')

Available Memory: 2.38 GB


In [8]:
data.describe()

,Unnamed: 0,hr,f_acc_x,f_acc_y,f_acc_z,f_gyr_x,f_gyr_y,f_gyr_z,f_loc_h,f_loc_v,...,w_acc_z,w_gyr_x,w_gyr_y,w_gyr_z,w_loc_h,w_loc_v,w_loc_d,w_mag_x,w_mag_y,w_mag_z
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1.717788e+12,111.486667,7.342569,-1.782610,5.642914,0.087608,-0.066490,0.101187,1.707605,1.780140,...,4.114459,0.002062,0.122406,0.065277,-3.338245,0.457550,12.888500,-16.788938,18.955688,-32.135064
std,2.901149e+03,0.611010,6.498912,5.812041,6.166445,1.149495,0.946380,1.178690,1.806069,0.226543,...,3.056653,1.024968,0.864068,0.906248,0.730429,0.751724,6.088267,14.199230,21.177506,7.209009
min,1.717788e+12,110.166667,-12.236304,-28.565548,-17.461040,-3.494615,-2.642447,-4.062407,0.165778,1.258523,...,-3.852444,-3.452455,-2.081980,-2.717737,-4.616870,0.000000,10.100000,-33.243752,-29.006250,-45.187500
25%,1.717788e+12,110.991667,5.618940,-2.222990,2.666255,-0.261482,-0.513053,-0.366268,0.335565,1.593669,...,1.909472,-0.549931,-0.272942,-0.234228,-3.957847,0.000000,10.100000,-26.685938,6.693750,-36.960939
50%,1.717788e+12,111.816667,7.084168,-1.157169,5.711598,0.004988,-0.218886,-0.062349,0.439643,1.978385,...,3.960121,-0.108887,-0.004734,0.121944,-2.981400,0.000000,10.100000,-20.231251,29.193751,-32.775002
75%,1.717788e+12,112.000000,10.269395,0.176071,7.450868,0.657338,0.209199,0.341038,4.268420,1.978385,...,7.186843,0.534087,0.447688,0.526336,-2.981400,0.987000,10.100000,-11.967188,33.046877,-27.375001
max,1.717788e+12,112.000000,22.357058,9.199420,27.372678,3.323709,2.897557,4.641439,4.268420,1.978385,...,8.925228,4.008495,3.790721,3.847074,-1.301601,1.920000,28.799999,29.643751,45.000000,-3.693750


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  100 non-null    int64  
 1   hr          100 non-null    float64
 2   f_acc_x     100 non-null    float64
 3   f_acc_y     100 non-null    float64
 4   f_acc_z     100 non-null    float64
 5   f_gyr_x     100 non-null    float64
 6   f_gyr_y     100 non-null    float64
 7   f_gyr_z     100 non-null    float64
 8   f_loc_h     100 non-null    float64
 9   f_loc_v     100 non-null    float64
 10  f_loc_d     100 non-null    float64
 11  f_mag_x     100 non-null    float64
 12  f_mag_y     100 non-null    float64
 13  f_mag_z     100 non-null    float64
 14  w_acc_x     100 non-null    float64
 15  w_acc_y     100 non-null    float64
 16  w_acc_z     100 non-null    float64
 17  w_gyr_x     100 non-null    float64
 18  w_gyr_y     100 non-null    float64
 19  w_gyr_z     100 non-null    fl

In [10]:
data.shape

(100, 27)

In [11]:
data=data.drop(columns=['Unnamed: 0'])

In [12]:
# 生成时间序列数据
def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i + sequence_length, :-1]
        # print(seq)
         # 最后一列是标签
        label = data.iloc[i + sequence_length - 1, -1]
        print(label)
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
sequence_length = 10
X, y = create_sequences(data, sequence_length)

In [14]:
# 将序列展平为二维数组
X_flat = X.reshape(X.shape[0] * X.shape[1], X.shape[2])

# 对展平后的数据进行标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat)

# 将数据重新调整为三维数组
X_scaled_reshaped = X_scaled.reshape(X.shape[0], X.shape[1], X.shape[2])


In [15]:
y

array(['Others', 'Others', 'Others', 'Others', 'Others', 'Others',
       'Others', 'Others', 'Others', 'Others', 'Others', 'Others',
       'Others', 'Others', 'Others', 'Others', 'Others', 'Others',
       'Others', 'Others', 'Others', 'Others', 'Others', 'Others',
       'Others', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Norm

In [16]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [17]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [18]:
memory_info = psutil.virtual_memory()
print(f'Available Memory: {memory_info.available / (1024 ** 3):.2f} GB')

Available Memory: 2.37 GB


In [19]:
# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [20]:
# 转换为PyTorch张量
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [21]:


# 创建DataLoader
batch_size = 1
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
memory_info = psutil.virtual_memory()
print(f'Available Memory: {memory_info.available / (1024 ** 3):.2f} GB')

Available Memory: 2.38 GB


In [23]:
# 参数设置
input_size = X_train.shape[2]
num_classes = len(np.unique(y))
num_channels = [25, 25, 25, 25]

In [24]:
input_size

25

In [25]:
num_classes

2

In [26]:
# 模型实例化
model = TCNClassifier(input_size, num_classes, num_channels)

/Users/minami/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [27]:



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 训练模型
num_epochs = 50


In [ ]:

for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        outputs = model(inputs.permute(0, 2, 1))
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.permute(0, 2, 1))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')